In [ ]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

In [ ]:
import collections
df = pd.read_csv('path_human_annotated_africanamercian_tweets')
dff = pd.read_csv('path_human_annotated_africanamercian_tweets2')
dfff = pd.read_csv('path_human_annotated_non-africanamercian_tweets')

In [ ]:
races = df['race'].to_list() + dff['race'].to_list() + dfff['race'].to_list()
tweets = df['tweet'].to_list() + dff['tweet'].to_list() + dfff['tweet'].to_list()

In [ ]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(tweets,races,stratify=races,test_size=0.25,random_state=1)

In [ ]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' #remove mentions
pat2 = r'https?://[^ ]+' #remove hyperlinks
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    lower_case = emoji.demojize(lower_case, language='en')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) #remove hashtag
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [ ]:
cleaned_tweets = []
for tweet in train_corpus:
    cleaned_tweets.append(tweet_cleaner_updated(tweet))

In [ ]:
uncleaned_tweets_test = test_corpus

In [ ]:
# use it when using data augmentation - EDA
with open('train_file_for_eda_human_annotated.txt','a') as file:
    for i in range(len(cleaned_tweets)):
        line = str(train_labels[i]) + '\t' + cleaned_tweets[i]
        file.write(line)
        file.write('\n')

In [ ]:
# use it when using data augmentation - EDA
new_races = []
train_tweets = []

with open(r'output_file_for_eda_human_annotated.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        race, tweet = line.split('\t')
        new_races.append(race)
        train_tweets.append(tweet.rstrip())

In [ ]:
cleaned_tweets_test = []
for tweet in test_corpus:
    cleaned_tweets_test.append(tweet_cleaner_updated(tweet))

In [ ]:
def tweet_cleaner_updated2(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    return stripped

In [ ]:
uncleaned_tweets = []
for tweet in train_corpus:
    uncleaned_tweets.append(tweet_cleaner_updated2(tweet))

In [ ]:
uncleaned_tweets_test = []
for tweet in test_corpus:
    uncleaned_tweets_test.append(tweet_cleaner_updated2(tweet))

In [ ]:
from advertools.emoji import extract_emoji
uncleaned_count_emoji = []
for tweet in uncleaned_tweets:
    count = 0
    for i in range(len(extract_emoji(tweet)['emoji_flat_text'])):
        count += 1
    uncleaned_count_emoji.append(count)

In [ ]:
uncleaned_count_emoji_test = []
for tweet in uncleaned_tweets_test:
    count = 0
    for i in range(len(extract_emoji(tweet)['emoji_flat_text'])):
        count += 1
    uncleaned_count_emoji_test.append(count)

In [ ]:
count_punctuation = lambda l1,l2: sum([1 for x in l1 if x in l2])
uncleaned_count_puncts = []
for tweet in uncleaned_tweets:
    uncleaned_count_puncts.append(count_punctuation(tweet,set(string.punctuation)))

In [ ]:
uncleaned_count_puncts_test = []
for tweet in uncleaned_tweets_test:
    uncleaned_count_puncts_test.append(count_punctuation(tweet,set(string.punctuation)))

In [ ]:
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}

code_pos_map = {v: k for k, v in  pos_code_map.items()}

In [ ]:
#abbrivation converters
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [ ]:
#POS tag converting
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [ ]:
pos_for_tweets = []
for tweet in cleaned_tweets:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets.append(text_pos(tweet))

In [ ]:
# use it when using data augmentation - EDA
pos_for_tweets = []
for tweet in train_tweets:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets.append(text_pos(tweet))

In [ ]:
pos_for_tweets_test = []
for tweet in cleaned_tweets_test:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets_test.append(text_pos(tweet))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v = TfidfVectorizer(stop_words = 'english', analyzer='char',ngram_range=(1, 1))
ngrams = v.fit_transform(pos_for_tweets)
ngrams_val = v.transform(pos_for_tweets_test)

In [ ]:
t = TfidfVectorizer(stop_words = 'english',max_features=10000,ngram_range=(1, 1))
tfidf_train = t.fit_transform(cleaned_tweets)
# use it when using data augmentation - EDA
#tfidf_train = t.fit_transform(train_tweets)

tfidf_val = t.transform(cleaned_tweets_test)

In [ ]:
print(type(tfidf_train))

In [ ]:
from scipy import sparse

In [ ]:
uncleaned_count_emoji_arr = np.array(uncleaned_count_emoji)
uncleaned_count_emoji_test_arr = np.array(uncleaned_count_emoji_test)
uncleaned_count_puncts_arr = np.array(uncleaned_count_puncts)
uncleaned_count_puncts_test_arr = np.array(uncleaned_count_puncts_test)

In [ ]:
uncleaned_count_emoji_arr = uncleaned_count_emoji_arr.reshape(5116,1)

In [ ]:
uncleaned_count_puncts_arr = uncleaned_count_puncts_arr.reshape(5116,1)

In [ ]:
uncleaned_count_emoji_test_arr = uncleaned_count_emoji_test_arr.reshape(1706,1)
uncleaned_count_puncts_test_arr = uncleaned_count_puncts_test_arr.reshape(1706,1)

In [ ]:
uncleaned_count_emoji_matrix = sparse.csr_matrix(uncleaned_count_emoji_arr)
uncleaned_count_emoji_test_matrix = sparse.csr_matrix(uncleaned_count_emoji_test_arr)
uncleaned_count_puncts_matrix = sparse.csr_matrix(uncleaned_count_puncts_arr)
uncleaned_count_puncts_test_matrix = sparse.csr_matrix(uncleaned_count_puncts_test_arr)

In [ ]:
tfidf_train.shape

In [ ]:
uncleaned_count_emoji_matrix.shape

In [ ]:
from scipy.sparse import hstack
train_train = hstack((tfidf_train,ngrams))
test_test = hstack((tfidf_val,ngrams_val))

In [ ]:
new_racess= []
for race in new_races:
    new_racess.append(int(race))
    

In [ ]:
from sklearn.svm import SVC

# use it when using data augmentation - EDA
clf2 = SVC(random_state=0).fit(train_train, new_racess)

y_pred = clf2.predict(test_test)
accuracy = accuracy_score(test_labels, y_pred)
accuracy

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

x_train1 = pd.Series(cleaned_tweets)
x_val1 = pd.Series(cleaned_tweets_test)

indexes = [i for i in range(5116)]
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(indexes, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result
all_x = pd.concat([x_train1,x_val1])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [ ]:
len(all_x_w2v)

In [ ]:
model_ug_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

In [ ]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.dv[prefix]
        n += 1
    return vecs

In [ ]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train1, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_val1, 100)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow, train_labels)

In [ ]:
clf.score(validation_vecs_dbow, test_labels)

In [ ]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha
    

In [ ]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train1, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_val1, 100)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc, train_labels)

In [ ]:
clf.score(validation_vecs_dmc,test_labels)

In [ ]:
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

In [ ]:
train_vecs_dmm = get_vectors(model_ug_dmm, x_train1, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_val1, 100)

In [ ]:
y_train = train_labels
y_validation = test_labels
clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)

In [ ]:
clf.score(validation_vecs_dmm, y_validation)

In [ ]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [ ]:
train_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_train1, 200)
validation_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_val1, 200)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmc, y_validation)

In [ ]:
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train1, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_val1, 200)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmm, y_validation)

In [ ]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser

In [ ]:
tokenized_train = [t.split() for t in cleaned_tweets]

In [ ]:
%%time
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)

In [ ]:
def labelize_tweets_bg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(bigram[t.split()], [prefix + '_%s' % i]))
    return result

In [ ]:
all_x_w2v_bg = labelize_tweets_bg(all_x, 'all')

In [ ]:
cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_x_w2v_bg)])

In [ ]:
%%time
for epoch in range(30):
    model_bg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dbow.alpha -= 0.002
    model_bg_dbow.min_alpha = model_bg_dbow.alpha

In [ ]:
train_vecs_dbow_bg = get_vectors(model_bg_dbow, x_train1, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_bg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_bg, y_validation)

In [ ]:
cores = multiprocessing.cpu_count()
model_bg_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmc.build_vocab([x for x in tqdm(all_x_w2v_bg)])

In [ ]:
%%time
for epoch in range(30):
    model_bg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmc.alpha -= 0.002
    model_bg_dmc.min_alpha = model_bg_dmc.alpha

In [ ]:
train_vecs_dmc_bg = get_vectors(model_bg_dmc, x_train1, 100)
validation_vecs_dmc_bg = get_vectors(model_bg_dmc, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmc_bg, y_train)

In [ ]:
clf.score(validation_vecs_dmc_bg, y_validation)

In [ ]:
cores = multiprocessing.cpu_count()
model_bg_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmm.build_vocab([x for x in tqdm(all_x_w2v_bg)])

In [ ]:
%%time
for epoch in range(30):
    model_bg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmm.alpha -= 0.002
    model_bg_dmm.min_alpha = model_bg_dmm.alpha

In [ ]:
train_vecs_dmm_bg = get_vectors(model_bg_dmm, x_train1, 100)
validation_vecs_dmm_bg = get_vectors(model_bg_dmm, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmm_bg, y_train)

In [ ]:
clf.score(validation_vecs_dmm_bg, y_validation)

In [ ]:
train_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_train1, 200)
validation_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_val1, 200)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_bg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmc_bg, y_validation)

In [ ]:
train_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_train1, 200)
validation_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_val1, 200)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_bg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmm_bg, y_validation)

In [ ]:
%%time
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

In [ ]:
def labelize_tweets_tg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result

In [ ]:
all_x_w2v_tg = labelize_tweets_tg(all_x, 'all')

In [ ]:
model_tg_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dbow.build_vocab([x for x in tqdm(all_x_w2v_tg)])

In [ ]:
%%time
for epoch in range(30):
    model_tg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dbow.alpha -= 0.002
    model_tg_dbow.min_alpha = model_tg_dbow.alpha

In [ ]:
train_vecs_dbow_tg = get_vectors(model_tg_dbow, x_train1, 100)
validation_vecs_dbow_tg = get_vectors(model_tg_dbow, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_tg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_tg, y_validation)

In [ ]:
cores = multiprocessing.cpu_count()
model_tg_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmc.build_vocab([x for x in tqdm(all_x_w2v_tg)])

In [ ]:
%%time
for epoch in range(30):
    model_tg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmc.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

In [ ]:
train_vecs_dmc_tg = get_vectors(model_tg_dmc, x_train1, 100)
validation_vecs_dmc_tg = get_vectors(model_tg_dmc, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmc_tg, y_train)

In [ ]:
clf.score(validation_vecs_dmc_tg, y_validation)

In [ ]:
cores = multiprocessing.cpu_count()
model_tg_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmm.build_vocab([x for x in tqdm(all_x_w2v_tg)])

In [ ]:
%%time
for epoch in range(30):
    model_tg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmm.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

In [ ]:
train_vecs_dmm_tg = get_vectors(model_tg_dmm, x_train1, 100)
validation_vecs_dmm_tg = get_vectors(model_tg_dmm, x_val1, 100)

In [ ]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dmm_tg, y_train)
clf.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
train_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_train1, 200)
validation_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_val1, 200)


In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_tg, y_train)
clf.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
train_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_train1, 200)
validation_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_val1, 200)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_tg, y_train)
clf.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train1, 200)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_val1, 200)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_ugdbow_tgdmm, y_train)

In [ ]:
clf.score(validation_vecs_ugdbow_tgdmm, y_validation)